In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from IPython.display import clear_output
import pickle

#common utilities for transforming data
from commons import Utilities
utils = Utilities()

In [2]:
DATA_DIR = "../input"

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
CHUNKSIZE = 50000

TRAIN_NUMERIC = "{0}/train_numeric.csv".format(DATA_DIR)
TRAIN_DATE = "{0}/train_date.csv".format(DATA_DIR)

TEST_NUMERIC = "{0}/test_numeric.csv".format(DATA_DIR)
TEST_DATE = "{0}/test_date.csv".format(DATA_DIR)


In [3]:
medians = pickle.load( open( "../exported/medians.p", "rb" ) )
models = pickle.load( open( "../exported/models.p", "rb" ) )

In [7]:
#Predict and write to file

submissions = []

def process(df_test,firstChunk,models):
    df_test = utils.normalize_data_w_medians(df_test,medians)
    df_test['Id2'] = df_test[ID_COLUMN]
    X_test = df_test.drop([ID_COLUMN],axis=1)
    inputnames = X_test.columns.values
    for model in models:
        X_test['p_0']=0
        X_test['p_1']=0
        X_test[ ['p_0','p_1'] ]  = model.predict_proba(X_test[inputnames])
        submission = pd.DataFrame({
            "Id":df_test[ID_COLUMN],
            "Response":X_test["p_1"]
        })
        sub_index=models.index(model)
        print(sub_index/len(models)*100)
        if(firstChunk):
            submissions.append(submission)
        else:
            submissions[sub_index]=pd.concat([submissions[sub_index],submission])

print("Evaluating test data...")
        
chunknum = 0
for chunk in pd.read_csv(TEST_NUMERIC, chunksize=CHUNKSIZE):
    process(chunk, chunknum == 0,models)
    chunknum = chunknum +1
    clear_output()
    print("%d percent" % int((chunknum*CHUNKSIZE/1183748)*100))

101 percent


In [8]:
pickle.dump( submissions, open( "../exported/submissions.p", "wb" ) )

In [12]:
i=0
column_names=[]
for df in submissions:
    column_name = TARGET_COLUMN+'_'+str(i)
    column_names.append(column_name)
    df.columns=[ID_COLUMN,column_name]
    i = i+1
    
final_submission = submissions[0]
for i in range(1,len(submissions)):
    final_submission = pd.merge(final_submission,submissions[i], on=ID_COLUMN, how='outer')

final_submission[TARGET_COLUMN] = final_submission[column_names].mean(axis=1)
final_submission[TARGET_COLUMN] = final_submission[TARGET_COLUMN].map(lambda x: int(1) if x>0.4 else int(0))
final_submission= final_submission.drop(column_names,axis=1)

with open('../submission.csv', 'a') as f:
    final_submission.to_csv(f, header=True, index=False)